# Kaggle-kilpailu: Gene Expression Prediction
### Pekko Ojanen, [pekkoo](https://www.kaggle.com/pekkoo) @ Kaggle
https://inclass.kaggle.com/c/gene-expression-prediction

## Taustaa
Kilpailu järjestettiin osana TTY:n SGN-41007 Pattern Recognition and Machine Learning -kurssia ja se oli avoinna myös kurssin ulkopuolelle. Tavoitteena oli ennustaa geeniekspressiotaso histonimuokkaussignaalien pohjalta. Geeniekspressiotasoja oli kaksi, korkea ja matala, joten kyseessä oli binäärinen luokitteluongelma. Kilpailun arviointikriteerinä toimi Area Under Curve (AUC), joten ennusteiden tuli olla korkean geeniekspressiotason todennäköisyyksiä.

Ryhmäämme "Group 40" kuuluivat lisäkseni Inkariina Simola ja Bahareh Darvishmohammadi. Päädyimme sijalle **4/125**. Jaoimme työt ryhmän kesken suurin piirtein siten, että Inkariina teki data-analyysia ja visualisointia etsien uusia featureita, Bahareh kokeili muutamia scikit-learniin implementoituja algoritmeja ja minä keskityin tunkkaamaan XGBoostia, neuroverkkoja sekä näistä ja muutamasta muusta mallista koostuvaa lopullista ensembleä.

Tässä raportissa esittelen vaihe vaiheelta siistityn koodini, mikä tuottaa neljännelle sijalle asettuvan ratkaisun ongelmaan. Suuri osa eksperimentoinnista on poistettu, jottei raportti olisi loputtoman pitkä. Myös koodisolujen tulosteet on poistettu, sillä kilpailun tuiskeessa soluja ajettiin epämääräisessä järjestyksessä, jolloin tulosteet olivat lopulta keskenään ristiriidassa.  Raportti etenee luontevasti datan esikäsittelystä ja featureiden luomisesta kohti lopullisen ensemblen syntymistä. Lopuksi pohditaan mitä kilpailusta opittiin ja mitä olisi kenties voinut tehdä paremmin.

## Koodi

Aluksi ladataan tarvittavat kirjastot ja tehdään muutamia asetuksia, jotka varmistavat tulosten toistettavuuden tai säätävät tulostuksia. 

In [ ]:
import os
import numpy as np
import pandas as pd

from bayes_opt import BayesianOptimization
import xgboost as xgb

# Scikit-learn
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.neighbors import KNeighborsClassifier

# Keras
from keras.models import Sequential
from keras.layers import Convolution1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization, LSTM
from keras.layers.advanced_activations import PReLU
from keras.callbacks import ModelCheckpoint

# Asetukset
pd.options.display.max_columns = 999
np.random.seed(123)
random_state = 2017
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)

Määritetään funktiot datan lataamiseen, skaalaamiseen sekä neuroverkkojen rakentamiseen. 

Funktio **load_data** tarjottiin professori Heikki Huttusen puolesta kilpailun foorumilla ja sitä on muokattu vain tekemällä datan uudelleenmuotoilu valinnaiseksi. Muokkaus johtuu siitä, että ratkaisussamme dataa tarvitaan kahdessa eri muodossa riippuen siitä syötetäänkö sitä neuroverkoille vai ei.

Neuroverkkojen arkkitehtuuri ja hyperparametrit etsittiin manuaalisesti kokeilemalla ja hieman [DeepChrome-arkkitehtuurista](https://arxiv.org/abs/1607.02078) inspiroituen. Kokeilua hankaloitti huomattavasti se, että neuroverkkoja pyöritettiin läppärillä ilman kunnollista näytönohjainta. Tämä oli melko hidasta ja uskoisin, että kunnollisella laskentateholla oltaisiin päädytty ainakin hieman erilaiseen lopputulokseen.

Funktio **rnn_model** rakentaa CNN+RNN-yhdistelmän, mikä saattaa näin jälkikäteen mietittynä olla tarpeettoman monimutkaista, sillä se toimii suunnilleen yhtä hyvin kuin pelkkä CNN-malli. Toisaalta se sisältää huomattavasti vähemmän parametreja kuin pelkkä CNN, mikä on suotavaa etenkin tässä tilanteessa, kun koulutusdataa ei ole kovin paljoa (15 485 observaatiota).

In [ ]:
def load_data(ravel=True):
    print("Loading data...")
    x_train = np.loadtxt("x_train.csv", delimiter = ",", skiprows = 1)
    x_test  = np.loadtxt("x_test.csv", delimiter = ",", skiprows = 1)    
    y_train = np.loadtxt("y_train.csv", delimiter = ",", skiprows = 1)
    
    print("All files loaded. Preprocessing...")

    # remove the first column(Id)
    x_train = x_train[:,1:]
    x_test  = x_test[:,1:]
    y_train = y_train[:,1:]

    # Every 100 rows correspond to one gene.
    # Extract all 100-row-blocks into a list using np.split.
    num_genes_train = x_train.shape[0] / 100
    num_genes_test  = x_test.shape[0] / 100

    print("Train / test data has %d / %d genes." % \
          (num_genes_train, num_genes_test))

    x_train = np.split(x_train, num_genes_train)
    x_test  = np.split(x_test, num_genes_test)

    if ravel:
        # Reshape by raveling each 100x5 array into a 500-length vector
        x_train = [g.ravel() for g in x_train]
        x_test  = [g.ravel() for g in x_test]
    
    # convert data from list to array
    x_train = np.array(x_train)
    y_train = np.array(y_train)
    x_test  = np.array(x_test)
    y_train = np.ravel(y_train)
    
    # Now x_train should be 15485 x 500 and x_test 3871 x 500.
    # y_train is 15485-long vector.
    
    print("x_train shape is %s" % str(x_train.shape))    
    print("y_train shape is %s" % str(y_train.shape))
    print("x_test shape is %s" % str(x_test.shape))
    print('Data preprocessing done...')
    
    return(x_train, y_train, x_test)

def minmax_scale(array, minimum=0, maximum=1):
    array_std = (array - array.min(axis=0)) / (array.max(axis=0) - array.min(axis=0))
    return array_std * (maximum - minimum) + minimum

def cnn_model():
    model = Sequential()
    model.add(Convolution1D(nb_filter=50, filter_length=10, border_mode='same',
                            input_shape=(100, 5)))
    model.add(PReLU())
    model.add(MaxPooling1D(5, border_mode='same'))
    model.add(Dropout(.3))
    model.add(BatchNormalization())
    
    model.add(Flatten())
    model.add(Dense(100))
    model.add(PReLU())
    model.add(Dropout(.3))
    model.add(BatchNormalization())
    
    model.add(Dense(100))
    model.add(PReLU())
    model.add(Dropout(.3))
    model.add(BatchNormalization())
    
    model.add(Dense(1, activation = 'sigmoid'))
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

def rnn_model():
    model = Sequential()
    model.add(Convolution1D(nb_filter=50, filter_length=10, border_mode='same',
                            input_shape=(100, 5)))
    model.add(PReLU())
    model.add(MaxPooling1D(5, border_mode='same'))
    model.add(Dropout(.4))
    model.add(BatchNormalization())
    
    model.add(LSTM(100))
    model.add(Dropout(.4))
    model.add(BatchNormalization())
    
    model.add(Dense(125))
    model.add(PReLU())
    model.add(Dropout(.2))
    model.add(BatchNormalization())
    
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

### Datan esikäsittely

Ladataan data 15485 x 500 -muodossa ja muutetaan dataframeiksi featureiden käsittelyn ja lisäämisen helpottamiseksi *pandas*-kirjaston avulla.

In [ ]:
x_train, y_train, x_test = load_data()
x_train_df = pd.DataFrame(x_train)
x_test_df = pd.DataFrame(x_test)

Ladataan data erikseen neuroverkoille soveltuvassa muodossa. Tällä kertaa muoto pysyy alkuperäisenä eikä tarvetta ole käsitellä featureita tai muuntaa dataa matriiseista dataframeiksi, sillä neuroverkkomme löytävät sopivat featuret itse. Neuroverkkojen oppimisen jouhevoittamiseksi data skaalataan välille 0 - 1.

In [ ]:
x_train_100_5, y_train, x_test_100_5 = load_data(ravel=False)
x_train_100_5 = minmax_scale(x_train_100_5)
x_test_100_5 = minmax_scale(x_test_100_5)

Luodaan funktio, mikä laskee statistiikkoja eri "markereille". Markerit esiintyvät dataframen kolumneissa viiden välein. Paljon muitakin statistiikkoja kokeiltiin (esim. mediaani, varianssi, minimi/maksimi ja summat), mutta nämä kolme muodostivat cross-validation-proseduurin perusteella parhaan joukon.

In [ ]:
def get_stats(row, index=0, stat='mean'):
    if stat == 'mean':
        return row[index:500:5].mean()
    elif stat == 'perc0': # Nollien prosentuaalinen osuus markerille
        return np.mean(row[index:500:5] == 0)
    elif stat == 'std':
        return np.std(row[index:500:5])
    else:
        raise ValueError(stat + ' currently not supported.')

Hyödyntäen funktiota yllä, luodaan kolumnit jokaiselle markerille jokaisesta statistiikasta sekä koulutus- että testausdataframeille.

In [ ]:
markers = ['H3K4me3', 'H3K4me1', 'H3K36me3', 'H3K9me3', 'H3K27me3']
dataframes = [x_train_df, x_test_df]
stats = ['mean', 'perc0', 'std']

for df in dataframes:
    for stat in stats:
        for i, j in enumerate(markers):
            df[j + '_' + stat] = df.apply(get_stats, args=(i, stat), axis=1)

### Ennustavien mallien rakentaminen

Luodaan DMatrix XGBoostia varten.

In [ ]:
xg_train = xgb.DMatrix(x_train_df, label=y_train)

Etsitään optimaalisia hyperparametreja XGBoostille Bayesilaisen optimoinnin avulla. Koodi on napattu käytännössä suoraan BayesianOptimization-kirjaston [Github-esimerkistä](https://github.com/fmfn/BayesianOptimization/blob/master/examples/xgboost_example.py).

In [ ]:
def xgb_evaluate(min_child_weight,
                 colsample_bytree,
                 max_depth,
                 subsample,
                 gamma,
                 alpha):

    params['min_child_weight'] = int(min_child_weight)
    params['colsample_bytree'] = max(min(colsample_bytree, 1), 0)
    params['max_depth'] = int(max_depth)
    params['subsample'] = max(min(subsample, 1), 0)
    params['gamma'] = max(gamma, 0)
    params['alpha'] = max(alpha, 0)

    cv_result = xgb.cv(params, xg_train, num_boost_round=num_rounds, 
                       nfold=5, seed=random_state, stratified=True, 
                       metrics='auc', callbacks=[xgb.callback.early_stop(100)])

    return cv_result['test-auc-mean'].values[-1]


num_rounds = 3000
num_iter = 120
init_points = 5
params = {
        'objective': 'binary:logistic',
        'eta': .01,
        'silent': 1,
        'verbose_eval': True,
        'seed': random_state
    }

xgbBO = BayesianOptimization(xgb_evaluate, {'min_child_weight': (1, 20),
                                            'colsample_bytree': (.1, 1),
                                            'max_depth': (1, 40),
                                            'subsample': (.6, 1),
                                            'gamma': (0, 8),
                                            'alpha': (0, 8),
                                            })

xgbBO.maximize(init_points=init_points, n_iter=num_iter)

Alla parhaat löydetyt hyperparametrit. Puista tuli huomattavan syviä.

In [ ]:
bayes_params3 = {
    'objective': 'binary:logistic',
    'eta': .01,
    'alpha': 0.9939,
    'colsample_bytree': .1066,
    'gamma': 1.8422,
    'max_depth': 39,
    'min_child_weight': 7,
    'subsample': .8101,
    'seed': random_state,
    'silent': 1
}

Rullataan XGBoostin oma cross-validation-proseduuri löydetyillä parametreilla, jolloin saadaan optimaalinen puiden lukumäärä.

In [ ]:
xgb_cv = xgb.cv(bayes_params_3, xg_train, num_boost_round=10000, early_stopping_rounds=200, nfold=5,
                stratified=True, verbose_eval=True, metrics='auc', seed=random_state)

Luodaan meta-versio koulutus- ja testausdataframeista ensemblen rakentamista varten. Lisätään näihin tyhjät kolumnit kaikille käytettäville malleille. Ensemble rakennetaan [Kagglen blogissa julkaistun ohjeen](http://blog.kaggle.com/2016/12/27/a-kagglers-guide-to-model-stacking-in-practice/) mukaisesti. Lopulliset käytettävät mallit valikoitiin rakentamalla erilaisia ensemblejä ja cross-validoimalla.

In [ ]:
x_train_df_meta = x_train_df.copy()
x_train_df_meta['XGB'] = np.nan
x_train_df_meta['CNN'] = np.nan
x_train_df_meta['RNN'] = np.nan
x_train_df_meta['LR'] = np.nan
x_train_df_meta['ET'] = np.nan
x_train_df_meta['RF'] = np.nan

x_test_df_meta = x_test_df.copy()
x_test_df_meta['XGB'] = np.nan
x_test_df_meta['CNN'] = np.nan
x_test_df_meta['RNN'] = np.nan
x_test_df_meta['LR'] = np.nan
x_test_df_meta['ET'] = np.nan
x_test_df_meta['RF'] = np.nan

Tehdään lista ensemblessä käytettävistä scikit-learn-algoritmeista ja niille annettavista nimistä. Jokaisen mallin hyperparametrit valittiin yksittäin cross-validoiden koulutusdatalla.

In [ ]:
clfs = [LogisticRegression(C=.01, penalty='l1'),
       ExtraTreesClassifier(n_estimators=800, n_jobs=2),
       RandomForestClassifier(n_estimators=800, n_jobs=2, criterion='entropy')]

clf_names = ['LR', 'ET', 'RF']

Sitten rakennetaan itse ensemble. Koulutusdata jaetaan viiteen osaan ja jokaisella kierroksella koulutetaan mallit neljällä osalla sekä ennustetaan korkean geeniekspressiotason todennäköisyydet viidennelle. Näitä ennustuksia tallennetaan jokaisella kierroksella koulutusdatan meta-version tietyille riveille, josta niitä myöhemmin käytetään koulutusdatana seuraavan tason mallissa.

Optimaalinen epochien määrä neuroverkoille tuntui vaihtelevan merkittävästi, joten mallit päädyttiin tallentamaan jokaiselta epochilta ja testaamaan niitä kaikkia kunkin kierroksen validointidatalla. Parhaat pisteet saava malli valittiin ja muut poistettiin. Tämä saattoi tarpeettomasti lisätä monimutkaisuutta, enkä ole varma oliko se nerokasta vai typerää.

Ensemblen rakentaminen kesti läppärilläni noin 14 tuntia.

In [ ]:
num_fold = 0
n_epochs = 25

CNN_test_preds = []
RNN_test_preds = []

neural_nets = ['CNN', 'RNN']

for train_index, test_index in cv.split(x_train_df, y_train):
    
    # Valitaan data sekä neuroverkoille että muille käytettäville malleille
    X_cvtrain = x_train_df.iloc[train_index]
    X_cvtest = x_train_df.iloc[test_index]
    X_cvtrain_100_5 = x_train_100_5[train_index]
    X_cvtest_100_5 = x_train_100_5[test_index]
    y_cvtrain = y_train[train_index]
    y_cvtest = y_train[test_index]
    
    num_fold += 1
    
    # Valmiiksi oltiin luotu viisi kansiota hakemistoon, yksi kutakin kierrosta varten
    os.chdir('/Users/peks/Documents/Studies/ML/Competition/Keras_models/Fold' + str(num_fold))
    
    # Koulutetaan molemmat neuroverkot loopissa, CNN ja CNN+RNN
    for net in neural_nets:
        if net == 'CNN':
            model = cnn_model()
        else:
            model = rnn_model()
            
        # Tallennetaan malli jokaiselta epochilta hakemistoon
        callbacks = [
            ModelCheckpoint(filepath='weights.-{epoch:02d}-{val_acc:.4f}.hdf5', monitor='val_acc')
        ]
        
        # Sovitetaan malli
        model.fit(X_cvtrain_100_5, y_cvtrain, validation_data=[X_cvtest_100_5, y_cvtest],
                  nb_epoch=n_epochs, callbacks=callbacks, batch_size=16)
        
        # Haetaan jokaisen epochin mallien nimet
        fold_model_names = os.listdir()[1:]
        epoch_aucs = np.array([])
        
        # Testataan jokaisen epochin mallia validointidatalla ja lisätään AUC-pisteytys listaan
        for epoch_model in fold_model_names:
            model.load_weights(epoch_model)
            epoch_auc = roc_auc_score(y_cvtest, model.predict(X_cvtest_100_5).ravel())
            epoch_aucs = np.append(epoch_aucs, epoch_auc)
        
        # Valitaan malleista parhaiten validointidatalla toimiva ja ladataan sen painotukset
        model.load_weights(fold_model_names[epoch_aucs.argmax()])
        
        # Poistetaan tallennetut epoch-mallit
        for file in fold_model_names:
            os.remove(file)
        
        # Ennustetaan sekä validointidatalla koulutus-metaa varten että 
        # testidatalla testi-metaa varten
        nn_fold_pred = model.predict(X_cvtest_100_5).ravel()
        x_train_df_meta.loc[test_index, net] = nn_fold_pred
        nn_test_pred = model.predict(x_test_100_5).ravel()
        
        # Listään testidatan ennusteet oikeaan listaan
        if net == 'CNN':
            CNN_test_preds.append(nn_test_pred)
        else:
            RNN_test_preds.append(nn_test_pred)
        
        # Tulostetaan kierroksen tulokset validointidatalla neuroverkoille
        print('Fold', num_fold, net, 'CV AUC:', roc_auc_score(y_cvtest, nn_fold_pred))
        
    # Rullataan scikit-learn mallit läpi ja ennustetaan jokaisella
    for i, clf in enumerate(clfs):
        clf.fit(X_cvtrain, y_cvtrain)
        clf_pred = clf.predict_proba(X_cvtest)[:, 1]
        x_train_df_meta.loc[test_index, clf_names[i]] = clf_pred
        
        print('Fold', num_fold, clf_names[i], 'CV AUC:', roc_auc_score(y_cvtest, clf_pred))
    
    # Koulutetaan XGBoost-malli aiemmin löydetyillä parametreilla
    xg_cvtrain = xgb.DMatrix(X_cvtrain, label=y_cvtrain)
    xg_cvtest = xgb.DMatrix(X_cvtest)
    
    xgb_model = xgb.train(bayes_params3, xg_cvtrain, num_boost_round=1184)
    xgb_cvpred = xgb_model.predict(xg_cvtest)
    x_train_df_meta.loc[test_index, 'XGB'] = xgb_cvpred
    print('Fold', num_fold, 'XGB CV AUC:', roc_auc_score(y_cvtest, xgb_cvpred))
    
    print('Fold', num_fold, 'completed.')
    print(50 * '-')

print('Train meta filled. Thank you.')
os.chdir('/Users/peks/Documents/Studies/ML/Competition/')

Tallennetaan tämänhetkinen koulutus-meta-dataframe CSV-tiedostoon varmuuden vuoksi.

In [ ]:
x_train_df_meta.to_csv('train_meta.csv', index=False)

Tarkistetaan CV-pisteet sovittamalla logistinen regressio meta-ennusteiden päälle.

In [ ]:
LR = LogisticRegression()
used_cols = ['XGB', 'CNN', 'RNN', 'LR', 'ET', 'RF']

cross_val_score(LR, x_train_df_meta[used_cols], y_train, scoring='roc_auc', cv=cv).mean()

Tutkitaan josko parantamisen varaa löytyisi lisäämällä alkuperäisiä marker-statistiikkoihin liittyviä featureita. Pyörittelemällä tätä hetken aikaa eri kombinaatioilla saatiin CV-pisteitä nostettua hieman.

In [ ]:
stat_cols = x_train_df_meta.columns[500:-7].values
used_cols = ['XGB', 'CNN', 'RNN', 'LR', 'ET','H3K27me3_std', 
             'H3K27me3_mean', 'H3K4me1_perc0', 'H3K9me3_std']

print('Baseline:', cross_val_score(LR, x_train_df_meta[used_cols], 
                      y_train, scoring='roc_auc', cv=cv).mean())

# Printataan uudet CV-pisteet featureita yksi kerrallaan lisäillen
for i in range(len(stat_cols)):
    print(stat_cols[i], cross_val_score(LR, x_train_df_meta[used_cols + [stat_cols[i]]],
                                        y_train, scoring='roc_auc', cv=cv).mean())

Aivan kilpailun viime hetkillä päätin testata vielä XGBoost-mallien rakentamista eri seedeillä ja näistä keskiarvon ottamista. Tämä paransi tulostamme vielä yllättävän paljon. Jos tämän olisi tajunnut tehdä aikaisemmin, olisin nostanut loopattavien seedien lukumäärää vielä hieman. Nyt tuli kiire niin piti rajoittaa.

In [ ]:
xgbs = []
xg_train = xgb.DMatrix(x_train_df, label=y_train)
xg_test = xgb.DMatrix(x_test_df)

for i in range(1, 8):
    complete_xgb_model = xgb.train(bayes_params3, xg_train, num_boost_round=int(1184/.8))
    xgb_pred = complete_xgb_model.predict(xg_test)
    xgbs.append(xgb_pred)
    
x_test_df_meta['XGB'] = np.mean(xgbs, axis=0)

Tehdään lopulliset yksittäisennusteet muilla scikit-learn-malleilla testidatalle.

In [ ]:
final_clfs = [LogisticRegression(C=.01, penalty='l1'),
              ExtraTreesClassifier(n_estimators=800, n_jobs=2)]
final_clf_names = ['LR', 'ET']

x_test_df_meta.drop('RF', axis=1, inplace=True) # Random forest huomattiin haitalliseksi ensemblessä
x_test_df_meta['CNN'] = np.mean(CNN_test_preds, axis=0)
x_test_df_meta['RNN'] = np.mean(RNN_test_preds, axis=0)

for i, clf in enumerate(final_clfs):
    clf.fit(x_train_df, y_train)
    x_test_df_meta[final_clf_names[i]] = clf.predict_proba(x_test_df)[:, 1]

Viimeisen tason mallina meta-featureiden päällä käytetään jälleen XGBoostia, jolle etsitään taas optimaalisia hyperparametreja Bayesilaisella optimoinnilla.

In [ ]:
xg_train_meta = xgb.DMatrix(x_train_df_meta[used_cols], label=y_train)
xg_test_meta = xgb.DMatrix(x_test_df_meta[used_cols])

In [ ]:
def xgb_evaluate(min_child_weight,
                 colsample_bytree,
                 max_depth,
                 subsample,
                 gamma,
                 alpha):

    params['min_child_weight'] = int(min_child_weight)
    params['colsample_bytree'] = max(min(colsample_bytree, 1), 0)
    params['max_depth'] = int(max_depth)
    params['subsample'] = max(min(subsample, 1), 0)
    params['gamma'] = max(gamma, 0)
    params['alpha'] = max(alpha, 0)

    cv_result = xgb.cv(params, xg_train_meta, num_boost_round=num_rounds, 
                       nfold=5, seed=random_state, stratified=True, 
                       metrics='auc', callbacks=[xgb.callback.early_stop(100)])

    return cv_result['test-auc-mean'].values[-1]


num_rounds = 3000
num_iter = 120
init_points = 5
params = {
        'objective': 'binary:logistic',
        'eta': .01,
        'silent': 1,
        'verbose_eval': True,
        'seed': random_state
    }

xgbBO = BayesianOptimization(xgb_evaluate, {'min_child_weight': (1, 20),
                                            'colsample_bytree': (.1, 1),
                                            'max_depth': (1, 40),
                                            'subsample': (.6, 1),
                                            'gamma': (0, 8),
                                            'alpha': (0, 8),
                                            })

xgbBO.maximize(init_points=init_points, n_iter=num_iter)

In [ ]:
bayes_params_final = {
    'objective': 'binary:logistic',
    'eta': .01,
    'alpha': .0748,
    'colsample_bytree': .8819,
    'gamma': .8596,
    'max_depth': 1,
    'min_child_weight': 2,
    'subsample': .6542,
    'seed': random_state,
    'silent': 1
}

Hyvät parametrit on löydetty, joten on aika tehdä lopulliset ennusteet.

In [ ]:
final_xgb_ = xgb.train(bayes_params_final, xg_train_meta, num_boost_round=int(912/.8))
xgb_pred = final_xgb_model.predict(xg_test_meta)

Luodaan data frame geenitunnisteista ja lopullisista ennusteista. Tallennetaan tämä CSV-tiedostona ja lähetetään kilpailuun. Lopullinen AUC-pisteytys on 0.92787 Kagglen yksityisellä pistetaululla.

In [ ]:
pred_df = pd.DataFrame({'GeneId': np.arange(1, x_test.shape[0] + 1), 
                        'Prediction': xgb_pred})
pred_df.to_csv('submission.csv', index=False)

## Mietteitä

Opin kilpailussa todella paljon ja osallistuminen oli lystiä. Ensimmäistä kertaa rakensin kunnon ensemblen, käytin Bayesilaista optimointia hyperparametrien löytämiseksi sekä käytin neuroverkkoja Kaggle-kilpailussa. Nähdäkseni parantamisen varaa jäi etenkin neuroverkkojen hyödyntämisen suhteen, sillä esimerkiksi [Kagglen foorumiketjussa](https://inclass.kaggle.com/c/gene-expression-prediction/forums/t/29637/congratulations-to-the-winners) toiseksi tulleet kertoivat käyttäneensä pelkästään CNN:ää, ottaen mediaanin 10-osaisen cross-validation-proseduurin ennusteista testidatalle. Neuroverkkojen tutkimista rajoitti kuitenkin pelkkä läppärin käyttö. Kaksi erilaista neuroverkkomallia tuli silti rakennettua, mikä oli luultavasti hieman ylitseampuvaa, vaikka se ensembleä tässä vähän kohensikin.

Parempien neuroverkkoarkkitehtuurien lisäksi olen lähes varma, että 10-osainen cross-validation olisi nostanut pisteitämme nykyisilläkin malleilla. Erityisesti se olisi uskoakseni hyödyttänyt juurikin neuroverkkoja, jotka olisivat nauttineet kasvaneesta koulutusdatan määrästä, ja joiden ennusteet testidatalla luotiin nimenomaan cross-validation-proseduurin aikana. Laskentatehon puutteen vuoksi käytin vain viittä osaa ristiinvalidoidessa. Myös XGBoostin seedejä olisi voinut rullata enemmän läpi, jos olisi sen aikasemmin tajunnut tehdä.

Neljänteen sijaan voi kuitenkin olla sangen tyytyväinen. Ero kolmanneksi ja toiseksi sijoittuneisiin joukkueisiin jäi pieneksi. Kilpailun voittaja sen sijaan paini täysin omassa sarjassaan. Ero muihin oli niinkin suuri, että haistelisimme voittajan löytäneen internetistä [oikeat vastaukset testidatalle](http://egg2.wustl.edu/roadmap/data/byDataType/rna/expression/).